# Esperanza's Data Re-cleaning Notebook

## Imports

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns

## Final Cleaning Work

In [ ]:
time_val = '1H'

In [ ]:
#col_list = ['time', 'Common Setpoint', 'Actual Sup Flow SP', 'Zone Temperature', 'Actual Supply Flow', 'Outside Air Temp', 'Humidity', 'energy']

floor_2 = pd.read_csv('../data/raw/merged_all_2F.csv', index_col = 'Unnamed: 0')
floor_3 = pd.read_csv('../data/raw/merged_all_3F.csv', index_col = 'Unnamed: 0')
floor_4 = pd.read_csv('../data/raw/merged_all_4F.csv', index_col = 'Unnamed: 0')

In [ ]:
floor_2 = floor_2.drop(['Unnamed: 0.1.1', 'Unnamed: 0.1'], axis = 1)

In [ ]:
floor_2['time_transformed'] = pd.to_datetime(floor_2['time'])
floor_2['time_floored'] = floor_2['time_transformed'].transform(lambda x: x.floor(freq=time_val))

In [ ]:
floor_2.head(5)

In [ ]:
floor_3 = floor_3.drop(['Unnamed: 0.1'], axis = 1)

In [ ]:
floor_3['time_transformed'] = pd.to_datetime(floor_3['time'])
floor_3['time_floored'] = floor_3['time_transformed'].transform(lambda x: x.floor(freq=time_val))

In [ ]:
floor_3.head(5)

In [ ]:
floor_4 = floor_4.drop(['Unnamed: 0.1'], axis = 1)

In [ ]:
floor_4['time_transformed'] = pd.to_datetime(floor_4['time'])
floor_4['time_floored'] = floor_4['time_transformed'].transform(lambda x: x.floor(freq=time_val))

In [ ]:
floor_4.head(5)

In [ ]:
grouped_df = pd.concat([floor_2, floor_3, floor_4])

In [ ]:
grouped_df.loc[:, 'hour'] = grouped_df['time_floored'].transform(lambda x: x.hour)

In [ ]:
split_date = "2018-08-01"

In [ ]:
# TRAIN TEST SPLIT:
train_set = grouped_df.loc[grouped_df['time_floored'] < split_date, :]
test_set = grouped_df.loc[~(grouped_df['time_floored'] < split_date), :]

In [ ]:
train_set.tail(5)

In [ ]:
# gap in July and August 2018 that this splits
# based on about 70% data split based on original data points

In [ ]:
test_set.head(5)

In [ ]:
train_meds = train_set.groupby(['time_floored']).median()
test_meds = test_set.groupby(['time_floored']).median()

In [ ]:
min_ts = train_meds.index[0]
max_ts = train_meds.index[len(train_meds) - 1]

In [ ]:
print(max_ts)

In [ ]:
missingtimes_df = pd.DataFrame(index = pd.date_range(min_ts, max_ts, freq=time_val))

In [ ]:
time_gaps = missingtimes_df.merge(train_meds, left_index = True, right_index = True, how = 'outer')
time_gaps['hour'] = time_gaps.index.hour

In [ ]:
hour_meds = train_set.groupby(['hour']).median()

In [ ]:
hour_meds.head(5)

In [ ]:
keep_cols = list(time_gaps.columns)[0: len(time_gaps.columns) - 1]
keep_cols_y = [x + "_y" for x in keep_cols]

In [ ]:
imputed_meds = time_gaps.loc[(time_gaps['energy'].isna()), :].merge(hour_meds, left_on = 'hour', right_index = True)

In [ ]:
imputed_meds = imputed_meds.loc[:, keep_cols_y + ["hour"]].rename({keep_cols_y[i]: keep_cols[i] for i in range(len(keep_cols))}, axis = 1)

In [ ]:
time_gaps.loc[(time_gaps['energy'].isna()), :] = imputed_meds

In [ ]:
sns.lineplot(data=time_gaps.reset_index(), x = 'index', y = 'energy')

In [ ]:
final_train = time_gaps.reset_index().rename({'index': 'time'}, axis = 1).drop(['hour'], axis = 1)

In [ ]:
final_test =test_set.drop(['hour', 'time', 'time_transformed'], axis = 1).rename({'time_floored': 'time'}, axis = 1)

In [ ]:
final_train.to_csv('training_set_imputed.csv', index = False)
final_test.to_csv('test_set_nonimputed.csv', index = False)

## Development/Idea Code for Cleaning

These rely on using the "Unnamed: 0.1.1" which look to be a remnant of when the data from the original 15 room values were used.

We ended up not following this because we could not verify it and we has trouble trying to reengineer rooms or figuring out how to impute values.

In [ ]:
# need to reimport to keep unnamed:0.1
floor_2 = pd.read_csv('../data/raw/merged_all_2F.csv', index_col = 'Unnamed: 0')
floor_3 = pd.read_csv('../data/raw/merged_all_3F.csv', index_col = 'Unnamed: 0')
floor_4 = pd.read_csv('../data/raw/merged_all_4F.csv', index_col = 'Unnamed: 0')

In [ ]:
floor_2['time_transformed'] = pd.to_datetime(floor_2['time'])
floor_2['time_floored'] = floor_2['time_transformed'].transform(lambda x: x.floor(freq=time_val))

floor_3['time_transformed'] = pd.to_datetime(floor_3['time'])
floor_3['time_floored'] = floor_3['time_transformed'].transform(lambda x: x.floor(freq=time_val))

floor_4['time_transformed'] = pd.to_datetime(floor_4['time'])
floor_4['time_floored'] = floor_4['time_transformed'].transform(lambda x: x.floor(freq=time_val))

In [ ]:
floor_2['floor'] = 2
floor_2['floor'] = 3
floor_2['floor'] = 4

In [ ]:
floor_2 = floor_2.drop(['Unnamed: 0.1.1'], axis = 1)

In [ ]:
grouped_df = pd.concat([floor_2, floor_3, floor_4])

### Using Means Instead of Medians

In [ ]:
group_means = grouped_df.groupby(['time_floored']).mean()
group_means.head(5)

In [ ]:
group_means.shape[0] * 0.7 // 1

In [ ]:
1 + group_means.shape[0] * 0.7 // 1

In [ ]:
train_set = group_means.iloc[0: int(group_means.shape[0] * 0.7 // 1)]
test_set = group_means.iloc[int(group_means.shape[0] * 0.7 // 1):]

In [ ]:
min_ts = train_set.index[0]
max_ts = train_set.index[len(train_set) - 1]

In [ ]:
df = pd.DataFrame(index = pd.date_range(min_ts, max_ts, freq=time_val))

In [ ]:
time_gaps = df.merge(train_set, left_index = True, right_index = True, how = 'outer')

In [ ]:
time_gaps.loc[time_gaps['energy'].isna()].shape

In [ ]:
mean_per_hour = train_set.groupby('hour').mean()

In [ ]:
mean_per_hour.head()

In [ ]:
# use these to fill NA values in the training_set

In [ ]:
sns.lineplot(data=train_set.reset_index(), x = 'time_floored', y = 'energy')

In [ ]:
# instead of splitting here, use earlier split dates for actually 30/70% data

In [ ]:
time_gaps.loc[:, 'hour'] = time_gaps.index.hour

In [ ]:
time_gaps.loc[time_gaps['energy'].isna()].shape

### Using Sums instead of Medians

In [ ]:
group_sums = grouped_df.groupby(['time_floored']).sum()
group_sums.head(5)

In [ ]:
group_sums.shape

In [ ]:
group_counts = grouped_df.groupby(['time_floored'])['Common Setpoint'].count().rename("count")

In [ ]:
group_counts.head(5)

In [ ]:
group_counts.min()

In [ ]:
group_counts.max()

In [ ]:
group_merge = group_sums.merge(group_counts, left_index = True, right_index = True)

In [ ]:
group_merge.head(5)

In [ ]:
group_merge = group_merge.drop(['Unnamed: 0.1', 'floor'], axis = 1)

In [ ]:
group_merge_mod = group_merge.multiply(group_counts.max() / group_merge['count'], axis = 'index')

In [ ]:
group_merge_mod.head(5)

In [ ]:
energy_cols = group_merge_mod.loc[:, ['energy', 'energy Next']]
all_else = group_merge_mod.drop(['energy', 'energy Next'], axis = 1)
all_else = all_else / group_counts.max()

In [ ]:
re_merge = all_else.merge(energy_cols, left_index = True, right_index = True)

In [ ]:
all_else['Actual Supply Flow'].isna().sum()

In [ ]:
sns.lineplot(data = all_else.reset_index(), x = 'time_floored', y = 'Actual Supply Flow')

In [ ]:
sns.lineplot(data = energy_cols.reset_index(), x = 'time_floored', y = 'energy')

In [ ]:
min_ts = all_else.index[0]
max_ts = all_else.index[len(all_else) - 1]

In [ ]:
df = pd.DataFrame(index = pd.date_range(min_ts, max_ts, freq=time_val))

In [ ]:
time_gaps = df.merge(re_merge, left_index = True, right_index = True, how = 'outer')

In [ ]:
time_gaps.loc[time_gaps['energy'].isna()].shape

In [ ]:
grouped_df['hour'] = grouped_df['time_floored'].transform(lambda x: x.time().hour)

In [ ]:
mean_per_hour = grouped_df.groupby('hour')['energy'].mean()

### Trying Different Versions of Filling/Imputing Null Values

In [ ]:
ffill_version = time_gaps.ffill()

In [ ]:
bfill_version = time_gaps.bfill()

In [ ]:
meanfill_version = time_gaps.fillna(time_gaps.mean())

In [ ]:
medfill_version = time_gaps.fillna(time_gaps.median())

In [ ]:
mixedfill = time_gaps.ffill(limit = 12).fillna(time_gaps.median())

In [ ]:
time_gaps.shape

In [ ]:
diff_ind = pd.date_range('2017-07-07 13:00:00+0000', '2019-01-07 23:00:00+0000', freq="1H").difference(all_else.index)


In [ ]:
len(diff_ind)

In [ ]:
diff_ind[0:5]

In [ ]:
pd.Series(diff_ind.hour).hist()

In [ ]:
time_gaps.head(1)

In [ ]:
sns.lineplot(data = time_gaps.reset_index(), x = 'index', y = 'energy')

In [ ]:
sns.lineplot(data = mixedfill.reset_index(), x = 'index', y = 'energy')

In [ ]:
sns.lineplot(data = ffill_version.reset_index(), x = 'index', y = 'energy')

In [ ]:
sns.lineplot(data = bfill_version.reset_index(), x = 'index', y = 'energy')

In [ ]:
sns.lineplot(data = meanfill_version.reset_index(), x = 'index', y = 'energy')

In [ ]:
sns.lineplot(data = medfill_version.reset_index(), x = 'index', y = 'energy')

In [ ]:
# that's another option - do the split before and after the dates with the biggest period of missiness?

In [ ]:
group_merge_mod['count'].value_counts()

In [ ]:
grouped_df_counts = grouped_df.groupby(['time_floored', 'Unnamed: 0.1'])['Common Setpoint'].count().rename("count")

In [ ]:
grouped_df_counts.max()

In [ ]:
grouped_df_counts.min()

In [ ]:
grouped_df_means = grouped_df.groupby(['time_floored', 'Unnamed: 0.1']).mean()
grouped_df_means.columns += ["_mean"]

In [ ]:
grouped_df_sums = grouped_df.groupby(['time_floored', 'Unnamed: 0.1']).sum()
grouped_df_sums.columns += ["_sum"]

In [ ]:
counts_sums = grouped_df_sums.merge(grouped_df_counts.to_frame(), left_index = True, right_index = True, how = 'outer')

In [ ]:
counts_sums = counts_sums.drop(['floor_sum'], axis = 1)

In [ ]:
count_factors = (9 / counts_sums['count'])

In [ ]:
counts_sums['factor'] = count_factors

In [ ]:
counts_sums.head()

In [ ]:
count_sums_mod = counts_sums.multiply(count_factors, axis = 'index')

In [ ]:
count_sums_mod['count'].value_counts()

In [ ]:
count_sums_mod.head(5)

In [ ]:
reset_sumsmod = count_sums_mod.reset_index()

In [ ]:
reset_sumsmod.loc[reset_sumsmod['time_floored'] == '2017-07-07 15:00:00+00:00']

In [ ]:
count_sums_mod.columns

In [ ]:
sums_avgs = count_sums_mod.merge(grouped_df_means, left_index = True, right_index = True)

In [ ]:
sums_avgs.head(5)

In [ ]:
count_sums_mod.shape

In [ ]:
sums_avgs.shape

In [ ]:
grouped_df_means.shape

In [ ]:
diff_ind = pd.date_range('2017-07-07 13:30:00+0000', '2019-01-07 23:30:00+0000', freq="30T").difference(count_sums_mod.index.get_level_values(0))

In [ ]:
len(diff_ind)

In [ ]:
diff_ind[0:5]

In [ ]:
pd.Series(diff_ind).plot()